Packages 

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)
import duckdb
from contextlib import contextmanager
from pathlib import Path

Data Understanding:
a. Initiale Exploration mit DuckDB

b. Datenübersicht erstellen
    Welche Variablen sind in den Tabellen enthalten?
    
    Gibt es fehlende Werte oder Ausreißer?
    
    Erste Verteilungen visualisieren (z. B. Einkommen vs. Energieverbrauch).

In [ ]:
# Define base path and data paths
BASE_PATH = Path('/Users/olli/Desktop/Uni/Predictive Analytics/Predictive-Analytics-Project/data/raw/predictive_analytics_data_testdb')
DATA_PATHS = {
    'electric_sensor': BASE_PATH / 'sensor_data/household_sensors_electric.csv',
    'gas_sensor': BASE_PATH / 'sensor_data/household_sensors_gas.csv',
    'home': BASE_PATH / 'metadata/home.csv',
    'person': BASE_PATH / 'metadata/person.csv',
    'weather': BASE_PATH / 'sensor_data/weather_complete.csv',
    'sensor_meta': BASE_PATH /  'metadata/sensor.csv',
    'sensorbox': BASE_PATH / 'metadata/sensorbox.csv',
    'survey_responses': BASE_PATH / 'survey_data/survey_responses.csv',
    'income_bands': BASE_PATH / 'survey_data/table_of_income_bands.csv'
}
    
def load_dataframes():
    """Load all CSV files into pandas DataFrames"""
    dataframes = {}
    
    for name, path in DATA_PATHS.items():
        if not path.exists():
            raise FileNotFoundError(f"File not found: {path}")
            
        try:
            df = pd.read_csv(path)
            dataframes[name] = df
            print(f"Loaded {name}: {df.shape[0]} rows, {df.shape[1]} columns")
        except Exception as e:
            print(f"Error loading {name}: {str(e)}")
    
    return dataframes

# Load all dataframes
dfs = load_dataframes()

# Access individual dataframes
electric_df = dfs['electric_sensor']
gas_df = dfs['gas_sensor']
home_df = dfs['home']
person_df = dfs['person']
weather_df = dfs['weather']
sensor_meta_df = dfs['sensor_meta']
sensorbox_df = dfs['sensorbox']
survey_df = dfs['survey_responses']
income_bands_df = dfs['income_bands']





Loaded electric_sensor: 1529499 rows, 11 columns
Loaded gas_sensor: 634755 rows, 11 columns
Loaded home: 255 rows, 24 columns
Loaded person: 616 rows, 13 columns
Loaded weather: 1160791 rows, 7 columns
Loaded sensor_meta: 20081 rows, 10 columns
Loaded sensorbox: 4033 rows, 24 columns
Loaded survey_responses: 416 rows, 169 columns
Loaded income_bands: 27 rows, 3 columns


1. Step: Explorative Datenanalyse/Data Understanding 
- Dataframe shape
- head and tail
- dtypes
- describe

In [31]:
# Home data
home_df.shape # (255, 24)
home_df.head()


,homeid,install_type,location,residents,starttime,starttime_enhanced,endtime,cohortid,income_band,study_class,hometype,equivalised_income,occupancy,urban_rural_class,urban_rural_name,build_era,new_build_year,smart_monitors,smart_automation,occupied_days,occupied_nights,entry_floor,outdoor_space,outdoor_drying
0,47,standard,Edinburgh,2,10/08/2016 00:00,NaN,08/05/2017 09:00,treatment_20170315,Missing,treatment,flat,missing,multiple,1,Large Urban Areas,1900-1918,NaN,Don't own,Don't own,2,7,3rd,No,NaN
1,59,standard,Edinburgh,2,06/10/2016 00:00,NaN,28/01/2018 07:56,control_20170315,"£90,000 or more",control,flat,above_median,multiple,1,Large Urban Areas,1900-1918,NaN,Don't own,Don't own,3,7,Ground,Yes - shared with neighbours,Yes
2,61,enhanced,Edinburgh,2,06/10/2016 00:00,17/11/2016 00:00,19/10/2017 07:00,enhanced_20170213,"£48,600 to £53,999",enhanced,house_or_bungalow,above_median,multiple,1,Large Urban Areas,1919-1930,NaN,Don't own,Don't own,2,7,Ground,Yes - shared with neighbours,Yes
3,62,enhanced,Edinburgh,2,26/08/2016 00:00,11/10/2016 00:00,30/06/2018 22:59,enhanced_20170213,"£43,200 to £48,599",enhanced,flat,above_median,multiple,1,Large Urban Areas,1850-1899,NaN,Don't own,Don't own,1,7,2nd,Yes - shared with neighbours,Yes
4,64,standard,Edinburgh,4,20/10/2016 00:00,NaN,30/06/2018 22:59,treatment_20170315,"£66,000 to £77,999",treatment,flat,above_median,multiple,1,Large Urban Areas,Before 1850,NaN,Don't own,Don't own,6,7,1st,Yes - shared with neighbours,Yes


In [32]:
home_df.columns

Index(['homeid', 'install_type', 'location', 'residents', 'starttime',
       'starttime_enhanced', 'endtime', 'cohortid', 'income_band',
       'study_class', 'hometype', 'equivalised_income', 'occupancy',
       'urban_rural_class', 'urban_rural_name', 'build_era', 'new_build_year',
       'smart_monitors', 'smart_automation', 'occupied_days',
       'occupied_nights', 'entry_floor', 'outdoor_space', 'outdoor_drying'],
      dtype='object')

In [33]:
home_df.dtypes

homeid                  int64
install_type           object
location               object
residents               int64
starttime              object
starttime_enhanced     object
endtime                object
cohortid               object
income_band            object
study_class            object
hometype               object
equivalised_income     object
occupancy              object
urban_rural_class      object
urban_rural_name       object
build_era              object
new_build_year        float64
smart_monitors         object
smart_automation       object
occupied_days           int64
occupied_nights         int64
entry_floor            object
outdoor_space          object
outdoor_drying         object
dtype: object

In [34]:
home_df.describe()

,homeid,residents,new_build_year,occupied_days,occupied_nights
count,255.000000,255.000000,17.000000,255.000000,255.000000
mean,198.901961,2.443137,2011.411765,3.964706,6.772549
std,80.249918,1.137973,4.691200,2.106124,0.857592
min,47.000000,0.000000,2002.000000,0.000000,0.000000
25%,133.500000,2.000000,2008.000000,2.000000,7.000000
50%,201.000000,2.000000,2013.000000,4.000000,7.000000
75%,267.500000,3.000000,2015.000000,6.000000,7.000000
max,335.000000,5.000000,2017.000000,7.000000,7.000000


In [35]:
income_bands_df.columns

Index(['data_release_value', 'response_options_initial',
       'response_options_mid'],
      dtype='object')